## Import library

In [17]:
import pandas as pd
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')
import test_results
import data_load
import numpy as np
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import evaluation as evl
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import sklearn as sk

from imblearn.over_sampling import SMOTE
from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

## Load data

In [2]:
df = pd.read_csv("/home/dtd/Downloads/Data_Scientist_Nanodegree/starbucks_portfolio_exercise/training.csv")

In [3]:
df.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [13]:
df['treatment'] = np.where(df['Promotion'] == 'Yes', 1, 0)

## Build model

In [14]:
def score(df, promo_pred_col = 'Promotion'):
    n_treat       = df.loc[df[promo_pred_col] == 'Yes',:].shape[0]
    n_control     = df.loc[df[promo_pred_col] == 'No',:].shape[0]
    n_treat_purch = df.loc[df[promo_pred_col] == 'Yes', 'purchase'].sum()
    n_ctrl_purch  = df.loc[df[promo_pred_col] == 'No', 'purchase'].sum()
    irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
    nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch
    return (irr, nir)

In [15]:
col = list(df.columns)

treatment = 'treatment' # intervention, or treatment
outcome = 'purchase' # outcome of interest

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove("ID")
cov.remove("Promotion")
print(cov)

features = col[:]
features.remove(outcome)
features.remove("ID")
features.remove("Promotion")
print(features)

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']
['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'treatment']


In [19]:
## Fit treatment
X = df[cov]
y = df[treatment]
ros = RandomOverSampler(random_state=42)
X_sampling, y_sampling = ros.fit_resample(X, y)
model_t = LogisticRegression()
model_t.fit(X_sampling, y_sampling)

## Fit outcome
X = df[features]
y = df[outcome]
ros = RandomOverSampler(random_state=42)
X_sampling, y_sampling = ros.fit_resample(X, y)
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 50)
model_y.fit(X_sampling, y_sampling)

GradientBoostingRegressor(n_estimators=50, random_state=0)

## Evaluation

In [31]:
def score(df, promo_pred_col = 'Promotion'):
    n_treat       = df.loc[df[promo_pred_col] == 'Yes',:].shape[0]
    n_control     = df.loc[df[promo_pred_col] == 'No',:].shape[0]
    n_treat_purch = df.loc[df[promo_pred_col] == 'Yes', 'purchase'].sum()
    n_ctrl_purch  = df.loc[df[promo_pred_col] == 'No', 'purchase'].sum()
    irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
    nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch
    return (irr, nir)

In [32]:
score(df, promo_pred_col = 'Promotion')

## Explore the changes in data

In [29]:
df['p1'] = model_t.predict_proba(df[cov].values)[:,:1]
df['p0'] = 1 - model_t.predict_proba(df[cov].values)[:,:1]

In [28]:
score(df, promo_pred_col = 'proposed_strategy')

In [38]:
rev_seq = []
dictionary_ = []
delta_result = []

df['proposed_strategy'] = ''

for i in tqdm(range(1, 1000)):
    delta = np.random.normal(1, i, len(df))
    df['q1'] = (delta * df['p1']) / (delta * df['p1'] + df['p0'])
    df['proposed_strategy'] = np.where(df['q1'] >= 0.5, 'Yes', 'No')
    a,b = score(df, promo_pred_col = 'proposed_strategy')
    rev_seq.append(b)

100%|██████████| 999/999 [00:36<00:00, 27.15it/s]


In [39]:
index_ = np.argmax(rev_seq)
print("Max revenue {}".format(rev_seq[index_]))


Max revenue -2234.0499999999993
